<a href="https://colab.research.google.com/github/chonholee/tutorial/blob/main/bigdata/BigdataII_Titanic_beginner_blank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titanic for Beginners
![](https://i.imgur.com/rRFchA8.png)

## 目次
0. ライブラリ・データ読み込み
1. データの概観・分析・前処理
2. 機械学習モデルの構築・学習
3. 予測の出力・提出

　このノートブックでは目次のように、データに最低限の整形を施して、機械学習モデルを学習し、予測を出力して提出するまでの流れを確認します。より本格的な取り組み方については、[Titanic Problem: A Complete Guide](https://www.kaggle.com/code/blurredmachine/titanic-survival-a-complete-guide-for-beginners)などを参照してください。

## 0. ライブラリ・データ読み込み  
　まず初めに使用するライブラリを読み込みます。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from pandas import DataFrame

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

　次にpandasのread_csv関数を用いて、分析する訓練データ**train.csv**とテストデータ**test.csv**を読み込みます。  
　変数名に与えた**df**は、**DataFrame**を意味しています（変数名は何でも構いません）。テストデータは素直にdf_testと命名したのに対して訓練データはdfとだけ命名したのは、後に説明するホールドアウト法やクロスバリデーションにおいて、さらにdfを擬似的な訓練データdf_trainと擬似的なテストデータdf_validに分割することを見越してのものです。\
※ファイルの読み込み方法は自身の作業場所によって変わります。自分の環境に合わせたコード片方を選択してください。両方または自分の環境外のコードを選択するとエラーが起こるので気をつけて下さい。

In [ ]:
# JupyterLabなどローカルで作業する場合
# 読み込むデータが格納されたディレクトリのパス，必要に応じて変更の必要あり
#パスを設定しましょう。****は各自データをアップロードしたフォルダの名前です。
path = "/content/drive/My Drive/****/"

df = pd.read_csv(path + 'titanic_train.csv')
df_test = pd.read_csv(path + 'titanic_test.csv')

## 1. データの概観・前処理

データから有用な知見を得るために、明確な目標があったほうが良いでしょう。

例えば、いくつかの具体的な問いを設定してみます。

1. タイタニック号の乗客はどのような人達だったのか？
2. それぞれの乗客はどのデッキにいたか？また、それは客室の種類とどのような関係にあったか？
3. 乗客は主にどこから来たのか？
4. 家族連れか、単身者か？

これらの基本的な問いの後に、さらに深くデータ解析を進めます。

5. 沈没からの生還者には、どのような要因があったのか？

[参照：初めてのKaggle](https://www.tsjshg.info/udemy/Lec56-59.html)

### 1.1 データの概観

データを見ていく上で、まず初めにデータのサイズを確認してみましょう。

In [ ]:
#*** here ***
print('訓練データのデータ数は{}、変数は{}種類です。'.format(    ))
print('テストデータのデータ数は{}、変数は{}種類です'.format(    ))

　訓練データの初めの10データを見てみましょう。

　変数名の一覧を見てみましょう。

　これらの変数名を、README.ipynbに示された変数の説明と対応付けておきましょう。  

変数 |定義 |備考  
---|---|---
Survived |死亡したかどうか |0 = No, 1 = Yes
Pclass |チケットのクラス |1 = 1st, 2 = 2nd, 3 = 3rd
Name |名前 |
Sex |性別 |
Age	|年齢 |
SibSp |乗船していた兄弟姉妹・配偶者の数	|
Parch |乗船していた親・子供の数	|
Ticket |チケット番号	|
Fare |チケット料金	|
Cabin |キャビン番号	|
Embarked |乗船した港	|C = Cherbourg, Q = Queenstown, S = Southampton



**補足：よく使う関数**

In [ ]:
# 有効値の数、データの型情報


In [ ]:
# 特定の項目を抽出
sub_df = df[['Survived','Pclass','Sex','Age']]
sub_df.head(5)

In [ ]:
# 統計量表示


In [ ]:
# 条件指定によるデータ抽出


In [ ]:
# ヒストグラム : 分布状況の確認に使用


In [ ]:
# ヒストグラムの重ね合わせ
df[df['Survived'] == 0]['Age'].hist(bins=20, alpha=0.3, color='red')
df[df['Survived'] == 1]['Age'].hist(bins=20, alpha=0.3, color='blue')

In [ ]:
# クロス集計


In [ ]:
# クロス集計結果の棒グラフ化
fig, ax = plt.subplots(1, 2, figsize=(12,4))
pd.crosstab(df['Pclass'], df['Survived']).plot.bar(ax=ax[0])
pd.crosstab(df['Survived'], df['Pclass']).plot.bar(ax=ax[1])

### 1.2 データの分析
　次に**EDA**と呼ばれる作業を行います。EDAとは、**Exploratory Data Analysis**の略で、日本語では**探索的データ分析**と訳されます。EDAでは、データを様々な角度から可視化したり、統計量を見ることで、データの特徴や構造を掴もうと試みます。この工程で得られた知見は機械学習モデルを選ぶ上でも、後に述べる特徴量エンジニアリングにおいても有用です。EDAで得た知見が役立つ理由の一つは、機械学習モデルによって仮定しているデータの特徴が異なることです。EDAによりデータに線型性・独立性・連続性などの特徴が観察できたり、後述の特徴量エンジニアリングでデータを加工することにより顕著な特徴を有した新しいデータを得ることができれば、それに適した機械学習モデルを用いることができます。  
　以下に行うEDAは、"EDA To Prediction (DieTanic)"というAshwini Swain氏によるKaggle Notebookを参考にしたものです。
  
EDA To Prediction (DieTanic)：https://www.kaggle.com/ash316/eda-to-prediction-dietanic

　まずは欠損値を確認しておきましょう。機械学習を用いたデータ分析に取り組む上で欠損値の確認は必須となっています。なぜならほとんどの機械学習モデルの実装は欠損値を含むデータに対して学習や予測ができず、エラーとなってしまうからです。

In [ ]:
# 学習データ


In [ ]:
# テストデータ


　**Age**、**Fare**, **Cabin**、**Embarked**の値の一部が欠損していることがわかりました。これらには後で対処することとします。  

　次に生存者の割合をみてみましょう。

In [ ]:
nb_survived = len(    )
nb_not_survived = len(    )

print ("Survived: %i (%.1f%%)"%(nb_survived, float(nb_survived)/len(df)*100.0))
print ("Not Survived: %i (%.1f%%)"%(nb_not_survived, float(nb_not_survived)/len(df)*100.0))
print ("Total: %i"%len(df))

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))

df['Survived'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Survived')
ax[0].set_ylabel('')

sns.countplot(x='Survived',data=df,ax=ax[1])
ax[1].set_title('Survived')
plt.show()

　生存率は38.4%であることがわかりました。分析対象となるデータには様々ありますが、一つの分類に**均衡データ**/**不均衡データ**というものがあります。不均衡データとは、主に予測対象のラベルの分布が著しく偏ったデータのことであり、病気の陽性/陰性などがその代表例です。不均衡データを分析する際には、データの前処理やモデルの構築、評価指標の選び方など様々な点において注意しなければなりません。しかし今回の予測対象であるPerishedは生存:死亡がおよそ4:6と均衡しているので、そうした心配の必要はありません。  

　次にデータの型について見てみましょう。機械学習を用いてデータ分析を行う際には、データの型にも注意が必要です。なぜならほとんどの機械学習モデルの実装はカテゴリカル変数を含むデータに対して学習や予測ができず、エラーとなってしまうからです。  
　データの型には大別して**数値データ**と**カテゴリカルデータ**があります。他にも日付・時間データなどがあったり、連続値データ/離散値データの区別があったりしますが、ここでは扱いません。数値データは文字通り数値が格納されたデータであり、カテゴリカルデータは主に文字列によってその分類が示されたデータです。ただしデータが数値であっても、その値の大小や順序が意味を持たない場合にはカテゴリカルデータとして扱う必要がある点には注意が必要です。  
　この観点では今回のデータは以下のように分類されます。
- 数値データ：Pclass, Age, SibSp, Parch, Fare
- カテゴリカルデータ：Name, Sex, Ticket, Embarked

　これらのカテゴリカルデータは機械学習モデルで扱えるよう、後で適切に処理します。

　ここからは一つ一つの変数について見ていきましょう。ただし、ここではデモンストレーションとして一部しか扱いません。またデータ分析コンペティションでは、必ずしも全てのEDAを自分で一から行う必要はありません。基本的なEDAは多くの場合Kaggle Notebookとして共有されますし、pandas-profilingなどの便利なライブラリを用いれば済んでしまうからです。しかし他の参加者との差別化を図るには、自らEDAで得た知見を活用する必要があります。また実務においてEDAを肩代わりしてくれる人はいません。これらの理由から、やはり自分である程度のEDAをこなせる必要はあるでしょう。

　まずは**Pclass**（チケットのクラス）について見ていきます。

In [ ]:
df.Pclass.value_counts()

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
df['Pclass'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'],ax=ax[0])
ax[0].set_title('Number of Passengers By Pclass')
ax[0].set_ylabel('Count')
sns.countplot(x='Pclass',hue='Survived',data=df,ax=ax[1])
ax[1].set_title('Pclass:Perished vs Survived')
plt.show()

　Pclassごとに人数および死亡率が著しく異なっていることが見て取れます。特にPclass=3は人数が圧倒的に多く、死亡率が著しく高いことがわかります。一方でPclass=1は死亡率が非常に低くなっています。Pclassはチケットのクラスでしたから、ここに見た事実は、Pclassの値が小さいほどチケットのグレードが高いことを直ちに示唆しています。他にはどのような知見が得られるか考えてみましょう。


　次に**Sex**（性別）について見てみます。

In [ ]:
sns.barplot(x='Sex', y='Survived', data=df)

In [ ]:
tab = pd.crosstab(df['Pclass'], df['Sex'])
print (tab)

# sum(1) means the sum across axis 1.
tab.div(tab.sum(1).astype(float), axis=0).plot(kind="bar", stacked=False)
plt.xlabel('Pclass')
plt.ylabel('Percentage')

　次に**Age**（年齢）について見てみます。

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
sns.violinplot(x="Pclass",y="Age", hue="Survived", data=df,split=True,ax=ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0,110,10))
sns.violinplot(x="Sex",y="Age", hue="Survived", data=df,split=True,ax=ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0,110,10))
plt.show()

　このような図を**バイオリン図**と言います。身近なところでは人口推計の男女別年齢分布が似たような図で示されています。この図からどのような知見が得られるでしょうか。最も顕著な傾向の一つは男性の幼年層に見られます。10歳以下の男性は死亡率が著しく低くなっています。この事実はタイタニック号の事故において幼い男の子が優先的に助けられたことを示唆しています。他にはどのような知見が得られるか考えてみましょう。


　最後に**相関行列**の**ヒートマップ**を表示してみましょう。相関行列とは各成分に対応する相関係数を並べた行列のことであり、値の大小に応じて色をつけたものをヒートマップと呼びます。この図を表示することによって、変数間の相関の強さを一目で把握することができます。

In [ ]:
sns.heatmap(df.corr(),annot=True,cmap='bwr',linewidths=0.2)
fig=plt.gcf()
fig.set_size_inches(10,8)
plt.show()

　この図から、**SibSpとParchの値に比較的強い正の相関がある**ことがわかります。SibSpは同乗していた兄弟姉妹・配偶者の数であり、Parchは同乗していた親・子供の数であったので、この事実は理解しやすいでしょう。ここでSibSpの値とParchの値の和をとって「同乗していた家族の人数」という新しい変数を加えるアイデアが得られます。なぜならSibSpとParchという不自然な分類で二つの変数に分割してあるよりも「同乗していた家族の人数」という変数の方が自然である可能性があるからです。

　他にも**PclassとFareの値に比較的強い負の相関**が見られます。この事実は、先に見たようにPclassの値が小さいほどチケットのグレードが高いという見立てを補強しています。この見立ては正しいと見ていいでしょう。  

　このように相関が強い変数がある場合には注意が必要です。相関の強い変数を機械学習モデルの学習に用いると、一部のモデルでは**多重共線性**という問題が生じます。そのため著しく相関の強い変数がある場合は、その変数のうち一つだけを残して他の変数を削除するといった対策をすることがあります。ここでは相関が強すぎるという程では無いと見て、こうした対策は行いませんが、自分で試してみても良いでしょう。

## 1.3 データの前処理
　ここでは、機械学習モデルが学習できるようにデータの前処理を行なっていきます。


　まずは**欠損値**の補完です。先に見たように**Age**、**Fare**, **Cabin**、**Embarked**の4変数は一部が欠損していました。欠損値の補完には様々な手法があります。平均値や最頻値といった代表値で補完する手法、機械学習モデルで予測して予測値で補完する手法、-9999などの外れ値で補完することによって欠損していたという情報を保持する手法などが挙げられます。
  
　このチュートリアルでは、欠損値を含む変数を削除します。欠損値のより良い取り扱い方については、他のサイト（[欠損値処理・補完をタイタニックデータを用いながら図解して理解しよう](https://qiita.com/MANGATA/items/9b988d031832230b9c3a)など）で詳しく学べます。

In [ ]:
missing_list = ['Age', 'Fare', 'Cabin', 'Embarked']

# データの削除
df.drop(missing_list, axis=1, inplace=True)
df_test.drop(missing_list, axis=1, inplace=True)

　次に**カテゴリカルデータ**を機械学習モデルで扱えるよう処理します。カテゴリカルデータには、**Name**, **Sex**, **Ticket**, **Embarked**がありました。  
　ここでも、カテゴリカルデータである変数を削除してしまいましょう。

In [ ]:
category_list = ['Name', 'Sex', 'Ticket']

df.drop(category_list, axis=1, inplace=True)
df_test.drop(category_list, axis=1, inplace=True)

**補足：欠損値の補間、カテゴリ変数の変換**

In [ ]:
temp = pd.read_csv(path + 'train.csv')

# 欠損値の確認
temp.isnull().sum()

In [ ]:
temp['Age'].head(10)

In [ ]:
temp['Age'] = temp['Age'].fillna(temp['Age'].mean())
temp['Age'].head(10)

In [ ]:
# カテゴリ変数の確認
temp['Sex'].head(10)

In [ ]:
temp['Sex'].map({'male':0, 'female':1})[:10]
temp['Sex'].head(10)

## 2. 機械学習モデルの構築・学習

　データが整形できたので、このデータを元に機械学習モデルを構築します。ここではロジスティック回帰というモデルを構築します。より本格的なモデル構築については、他のサイト（[KaggleのTitanicでモデルを選別する](https://qiita.com/sudominoru/items/1c21cf4afaf67fda3fee)など）を参照してください。
  
　まずdfとdf_testを**説明変数**と**目的変数**に分けます。
- 説明変数：モデルの学習に使用する変数、今回の問題ではPassengerId, Survived以外の変数
- 目的変数：予測対象の変数, 今回の問題ではSurvived  

　ここでスライスしたdfとdf_testを.valuesとしてnumpy.ndarray型に変換しているのは、機械学習モデルの実装によってはこの型のデータしか受け付けないからです。

In [ ]:
X =
y =

X_test =

　機械学習モデルにとって最大の障害の一つは**過学習**です。過学習とは機械学習モデルが訓練データを学習する際に、訓練データに対して正しい予測を与えようとするあまり、訓練データにしか良い予測を与えられず、テストデータや他のデータに対して役に立たなくなってしまう現象のことです。

　この現象を回避するための手法の一つに**ホールドアウト法**があります。ホールドアウト法では、与えられた訓練データをさらに擬似訓練データと擬似テストデータに分割し、機械学習モデルを擬似訓練データで学習させます。その上で、擬似訓練データに対する予測精度と擬似テストデータに対する予測精度を比較して、二つの値に大きな解離が見られる場合には過学習が発生していると判断し、過学習を抑えるよう修正を加えます。  

　今回は7:3で元の訓練データを分割して、擬似訓練データ(X_train, y_train)と擬似テストデータ(X_valid, y_valid)とします。変数名は何でも構いませんが、ここで用いたvalidとはvalidation(検証)の略です。これは擬似テストデータをモデルの予測精度の検証に用いることに由来します。

　データの分割には、scikit-learnのtrain_test_split関数を使用しますが、分割はランダムに行われるため、再現性を保つためには乱数生成のシード値を引数random_stateで指定する必要があります。この値を42とする例が海外を中心に散見されるのは、この数字が、有名なSF作品「銀河ヒッチハイク・ガイド」で「生命、宇宙、そして万物についての究極の疑問の答え」とされているからだそうです。

　ホールドアウト法の拡張には、**クロスバリデーション**があります。クロスバリデーションについては、[Kaggle Titanic Tutorial](https://www.kaggle.com/code/sashr07/kaggle-titanic-tutorial)で解説しています。

In [ ]:
X_train, X_valid, y_train, y_valid =

　ロジスティック回帰モデルを作成して、擬似訓練データ(X_train, y_train)を学習させます。

In [ ]:
lr =

lr.fit(X_train, y_train)

　このモデルによる予測精度の評価を、今回のコンペティションで指定された評価基準である**正解率(accuracy)**で行います。先述したように、擬似訓練データ(X_train, y_train)に対するスコアと擬似テストデータ(X_valid, y_valid)に対するスコアを見ます。これらの値が著しく解離している場合には、**過学習**が発生しているとして修正を行います。

In [ ]:
print('Train Score: {}'.format(round(lr.score(X_train, y_train), 3)))
print(' Test Score: {}'.format(round(lr.score(X_valid, y_valid), 3)))

補足：ランダムフォレストの例

## 3. 予測の出力・提出
　学習させたロジスティック回帰モデルを用いて、テストデータに対する予測を行います。

In [ ]:
y_pred =
y_pred

　このようにして提出すべき予測値が得られました。

　最後に得られた予測値を規定の形式に整形して、csvファイルとして出力しましょう。  
　まず規定の形式を確認しましょう。以下のようなcsvファイルで提出するよう指示されていました。

PassengerID|Survived
---|---
892|0
893|1
894|0
…|…
1307|0
1308|0
1309|0

　また、gender_submission.csvがその例とされていたので、これを確認します。\
※ファイルの読み込み方法は自身の作業場所によって変わります。自分の環境に合わせたコード片方を選択してください。両方または自分の環境外のコードを選択するとエラーが起こるので気をつけて下さい。

In [ ]:
 # JupyterLabなどローカルで作業する場合
 # 読み込むデータが格納されたディレクトリのパス，必要に応じて変更の必要あり
 #パスを設定しましょう。****は各自データをアップロードしたフォルダの名前です。
path = "/content/drive/My Drive/******/"

submission = pd.read_csv(path + 'gender_submission.csv')
submission

In [ ]:
submission.head(10)

　提出ファイルを作成するには、このデータフレームのSurvivedを上書きするのが手っ取り早いでしょう。

In [ ]:
submission['Survived'] = y_pred
submission.head(10)

　これをcsvファイルとして出力すれば、提出ファイルの完成です。\
※csvファイル書き出しの方法は自身の作業場所によって変わります。自分の環境に合わせたコード片方を選択してください。両方または自分の環境外のコードを選択するとエラーが起こります。

In [ ]:
# JupyterLabなどローカルで作業する場合
# パスは必要に応じて変更の必要あり
submission.to_csv('/content/drive/My Drive/Lecture_BigData/Kaggle-Titanic/submission.csv', index=False)

In [ ]:
# Google Drive・Google Colaboratoryで作業する場合

from google.colab import files
# colaboratory上に保存
# 保存したcsvファイルはランタイムが終了すると削除されます
submission.to_csv('submission.csv', index=False)
# colaboratory上に保存したcsvファイルをローカルに保存
files.download('submission.csv')

**補足：ランダムフォレストを使用した例**

In [ ]:
import re
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

df2 = pd.read_csv(path + 'train.csv')
df2_test = pd.read_csv(path + 'test.csv')

# 前処理
def preprocess(df):
    df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Embarked'] = df['Embarked'].fillna('Unknown')
    df['Sex'] = df['Sex'].apply(lambda x: 1 if x == 'male' else 0)
    df['Embarked'] = df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2, 'Unknown': 3} ).astype(int)
    df = df.drop(['Cabin','Name','PassengerId','Ticket'],axis=1)
    return df

# 学習
def train(df):
    train_x = df.drop('Survived', axis=1)
    train_y = df.Survived
    (train_x, test_x ,train_y, test_y) = train_test_split(train_x, train_y, test_size = 0.33, random_state = 42)

    clf = RandomForestClassifier(random_state=0)
    clf = clf.fit(train_x, train_y)
    pred = clf.predict(test_x)
    print("Accuracy: ", accuracy_score(pred, test_y))

    features = train_x.columns
    importances = clf.feature_importances_
    indices = np.argsort(importances)
    for i in indices[::-1]:
        print("{:<15} {:f}%".format(features[i], importances[i]*100))
    return clf

df2 = preprocess(df2)
clf = train(df2)